In [3]:
class Server:
    CURR_NUMBER = 0

    def __new__(cls):
        cls.CURR_NUMBER += 1
        return super().__new__(cls)
        
    def __init__(self):
        self.buffer = []
        self.routers = []
        self.ip = self.CURR_NUMBER
        
    def send_data(self, data):
        for router in self.routers:
            router.buffer.append(data)

    def get_data(self):
        curr_buffer = self.buffer.copy()
        self.buffer = []
        return curr_buffer

    def get_ip(self):
        return self.ip

class Router:
    def __init__(self):
        self.buffer = []
        self.servers = {}
        
    def link(self, server):
        self.servers[server.ip] = server
        server.routers.append(self)

    def unlink(self, server):
        s = self.servers.pop(server.ip, False)

        if s:
            server.routers.pop(server.routers.index(self))

    def send_data(self):
        for package in self.buffer:
            if package.ip in self.servers:
                self.servers[package.ip].buffer.append(package)

        self.buffer = []

class Data:
    def __init__(self, data, ip):
        self.data = data
        self.ip = ip

In [4]:
router = Router()
sv_from = Server()
sv_from2 = Server()
router.link(sv_from)
router.link(sv_from2)
router.link(Server())
router.link(Server())
sv_to = Server()
router.link(sv_to)
sv_from.send_data(Data("Hello", sv_to.get_ip()))
sv_from2.send_data(Data("Hello", sv_to.get_ip()))
sv_to.send_data(Data("Hi", sv_from.get_ip()))
router.send_data()
msg_lst_from = sv_from.get_data()
msg_lst_to = sv_to.get_data()